In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

from sklearn.utils import compute_class_weight

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ReduceLROnPlateau

import keras
from keras import regularizers
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
from tensorflow.keras.applications import VGG19


In [2]:
data_dir = '/Users/marta/Documents/data_dir/'

train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(256, 256),
  batch_size = 64)

val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(256, 256),
  batch_size = 64)

Found 14633 files belonging to 2 classes.
Using 11707 files for training.
Found 14633 files belonging to 2 classes.
Using 2926 files for validation.


In [3]:
# Helped by Argishti Ovsepyan 
classes = np.array(['PLP', 'POR'])
y = [classes[0]] * 5422 + [classes[1]] * 9211
class_weights = compute_class_weight('balanced', classes=classes, y=y)

class_weights_dict = class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
class_weights_dict

{0: 1.349409811877536, 1: 0.794322006296819}

In [4]:
# Define the learning rate scheduler
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)


In [9]:
from tensorflow.keras.layers import Dropout

# Add dropout layers
cnn5 = Sequential()

cnn5.add(Conv2D(512, 3, activation='relu', input_shape=(256, 256, 3)))
cnn5.add(MaxPooling2D(2, padding='same'))
cnn5.add(Dropout(0.5))

cnn5.add(Conv2D(256, 3, activation='relu', kernel_regularizer=regularizers.l2(0.03)))
cnn5.add(MaxPooling2D(2, padding='same'))
cnn5.add(Dropout(0.5))

cnn5.add(Conv2D(256, 3, activation='relu', kernel_regularizer=regularizers.l2(0.03)))
cnn5.add(MaxPooling2D(2, padding='same'))
cnn5.add(Dropout(0.5))

cnn5.add(Conv2D(256, 3, activation='relu', kernel_regularizer=regularizers.l2(0.03)))
cnn5.add(MaxPooling2D(2, padding='same'))
cnn5.add(Dropout(0.5))

cnn5.add(Flatten())
cnn5.add(Dense(1, activation='sigmoid'))

cnn5.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), metrics=['acc'])

In [ ]:
history5 = cnn5.fit(train_ds, epochs=20, validation_data=val_ds, class_weight=class_weights_dict, callbacks=[reduce_lr])

Epoch 1/20


2024-06-12 09:13:19.835194: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_17' with dtype double and shape [2]
	 [[{{node Placeholder/_17}}]]
2024-06-12 09:13:19.835451: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_17' with dtype double and shape [2]
	 [[{{node Placeholder/_17}}]]


183/183 [==============================] - ETA: 0s - loss: 24.7043 - acc: 0.5303 

2024-06-12 10:49:21.300827: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2926]
	 [[{{node Placeholder/_4}}]]
2024-06-12 10:49:21.301880: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [2926]
	 [[{{node Placeholder/_4}}]]


183/183 [==============================] - 6134s 34s/step - loss: 24.7043 - acc: 0.5303 - val_loss: 14.7167 - val_acc: 0.6237 - lr: 0.0010
Epoch 2/20
183/183 [==============================] - 6298s 34s/step - loss: 11.6314 - acc: 0.5724 - val_loss: 9.1202 - val_acc: 0.6193 - lr: 0.0010
Epoch 3/20
183/183 [==============================] - 5567s 30s/step - loss: 7.4746 - acc: 0.5815 - val_loss: 6.0685 - val_acc: 0.6203 - lr: 0.0010
Epoch 4/20
150/183 [=======================>......] - ETA: 18:36 - loss: 5.2650 - acc: 0.5939

In [ ]:
cnn5.save('./saved_models/cnn5.h5')
pd.DataFrame(history5.history).to_csv('./saved_models//history5.csv', index=False)

In [ ]:
#### Check out the plot of loss vs epoch.
plt.figure(figsize = (12, 6));

plt.subplot(1,2,1)
plt.plot(history5.history['loss'], c = 'navy', label = 'Training Loss');
plt.plot(history5.history['val_loss'], c = 'orange', label = 'Testing Loss');

plt.title('''CNN 5 :
Binary Crossentropy (loss function),
as a Function of Epochs''')
plt.xlabel('Epochs');
plt.ylabel('Loss Function')
plt.legend();


plt.subplot(1, 2, 2)
plt.plot(history5.history['acc'], c = 'navy', label = 'Training Accuracy');
plt.plot(history5.history['val_acc'], c = 'orange', label = 'Testing Accuracy');
plt.title('''CNN 5: 
Accuracy Score 
as a Function of Epochs)''')
plt.xlabel('Epochs');
plt.ylabel('Accuracy Score')
plt.legend();